# Imports

In [ ]:
from riotwatcher import LolWatcher
import riotwatcher
import cassiopeia as cass
import pandas as pd
import numpy as np
import json 
import time
import timeit

# Añadir key

In [ ]:
key = "RGAPI-80b2497f-650f-4c41-b981-898efd21a8b6"
watcher = LolWatcher(key)
cass.set_riot_api_key(key)

# Obtener builds a partir de partidas

Lee una lista de ficheros y realiza las builds a partir de estos:

In [ ]:
def get_builds_lanes():
    builds = {}
    match_counter = 0
    high_elo = 0
    
    #Items que no queremos contar
    banned_items = ['1054', '1055', '1056', '2055', '1001', '1102','1103','1101','2003','2010','3513','2139','2138','2140','3853','3860','3864','3857','7004','3851','3863','3855','3859','3858','3862','3854','3850','2403','3400','1083','2033','7003']
    #Elos que consideramos """medio-alto""" entre muchas comillas.
    elos = ['CHALLENGER', 'GRANDMASTER', 'MASTER', 'DIAMOND', 'PLATINUM', 'GOLD', 'SILVER'] 
    #Archivos con partidas
    files = ['data_matchPROCESSED_1.json', 'data_matchPROCESSED_2.json', 'data_matchPROCESSED_3.json', 'data_matchPROCESSED_4.json', 'data_matchPROCESSED_5.json', 'data_matchPROCESSED_6.json','data_matchPROCESSED_7.json', 'data_matchPROCESSED_8.json', 'data_matchPROCESSED_9.json']
    
    #Info de items
    items = watcher.data_dragon.items('13.7.1')['data']
    
    for file in files:
        print("Opening:", file, "...")
        f = open(file)
        matchinfo = json.load(f)
        f.close()
        print("Processing", file)
        matches = list(matchinfo.values())
        for match in matches:
            match_counter += 1
            #Comprobamos el elo
            if match[1] in elos:
                high_elo += 1
                info = match[0]
                #Por cada champ miramos su información
                for player in info['info']['participants']:
                    champId = player['championId']
                    champName = player['championName']
                    role = player['individualPosition']
                    firstLane = False
                    secondLane = False
                    for champ in champs:
                        if champId == champ['id']:
                            champInfo = champ
                            break
                    if champId not in builds.keys():
                        builds[champId] = {"name": champName, "lane" : {"global": {"mythic": [], "boots": [], "legendary": [], "trinket": []}, "firstLane": None, "secondLane": None}}
                    
                    #Traducimos los roles
                    if role == 'TOP':
                        role = 'TOP'
                    if role == 'MIDDLE':
                        role = 'MID'
                    if role == 'UTILITY':
                        role = 'SUP'
                    if role == 'JUNGLE':
                        role = 'JGL'
                    if role == 'BOTTOM':
                        role = 'ADC'
                    #Comprobamos si son primarios o secundarios
                    if role == champInfo['main_role']:
                        firstLane = True
                    if role == champInfo['secondary_role']:
                        secondLane = True
                    
                    if firstLane and builds[champId]["lane"]['firstLane'] == None:
                        builds[champId]["lane"]['firstLane'] =  {"mythic": [], "boots": [], "legendary": [], "trinket": []}
                    if secondLane and builds[champId]["lane"]['secondLane'] == None:
                        builds[champId]["lane"]['secondLane'] =  {"mythic": [], "boots": [], "legendary": [], "trinket": []}
                    
                    #Iteramos items
                    for item_index in range(7):
                        item = 'item'+str(item_index)
                        item_id = str(player[item])
                        if item_id != '0' and item_id not in banned_items:
                            item_data = items[item_id]
                            #Si es un mejorado de Ornn ponemos la versión de tienda
                            if 'requiredAlly' in item_data:
                                if item_data['requiredAlly'] == 'Ornn':
                                    item_id = item_data['from'][0]
                                    item_data = items[item_id]
                            #Miramos el tipo y añadimos a global y la linea que corresponda
                            if ("rarityMythic" in item_data['description']):
                                builds[champId]["lane"]['global']["mythic"].append(item_id)
                                if firstLane:
                                    builds[champId]["lane"]['firstLane']["mythic"].append(item_id)
                                if secondLane:
                                    builds[champId]["lane"]['secondLane']["mythic"].append(item_id)
                            elif ("Boots" in item_data['tags']):
                                builds[champId]["lane"]['global']["boots"].append(item_id)
                                if firstLane:
                                    builds[champId]["lane"]['firstLane']["boots"].append(item_id)
                                if secondLane:
                                    builds[champId]["lane"]['secondLane']["boots"].append(item_id)
                            elif ("Trinket" in item_data['tags']):
                                builds[champId]["lane"]['global']["trinket"].append(item_id)
                                if firstLane:
                                    builds[champId]["lane"]['firstLane']["trinket"].append(item_id)
                                if secondLane:
                                    builds[champId]["lane"]['secondLane']["trinket"].append(item_id)
                            elif ("into" not in item_data):
                                builds[champId]["lane"]['global']["legendary"].append(item_id)
                                if firstLane:
                                    builds[champId]["lane"]['firstLane']["legendary"].append(item_id)
                                if secondLane:
                                    builds[champId]["lane"]['secondLane']["legendary"].append(item_id)
                            
    #Contamos los items y ordenamos por uso           
    for i in list(builds.keys()):
        for j in list(builds[i]["lane"].keys()):
            if builds[i]['lane'][j] != None:
                build_tosort = list(builds[i]["lane"][j].values())
                for h in range(4):
                    item_list = np.unique(build_tosort[h], return_counts=True)
                    if h == 0:
                        index_sort = np.argsort(item_list[1])[::-1]
                        builds[i]["lane"][j]["mythic"] = list(zip(item_list[0][index_sort].tolist(), item_list[1][index_sort].tolist()))
                    if h == 1:
                        index_sort = np.argsort(item_list[1])[::-1]
                        builds[i]["lane"][j]["boots"] = list(zip(item_list[0][index_sort].tolist(), item_list[1][index_sort].tolist()))
                    if h == 2:
                        index_sort = np.argsort(item_list[1])[::-1]
                        builds[i]["lane"][j]["legendary"] = list(zip(item_list[0][index_sort].tolist(), item_list[1][index_sort].tolist()))
                    if h == 3:
                        index_sort = np.argsort(item_list[1])[::-1]
                        builds[i]["lane"][j]["trinket"] = list(zip(item_list[0][index_sort].tolist(), item_list[1][index_sort].tolist()))
    
    print("From", match_counter, "matches,", high_elo, "were high elo.")
    return builds    

In [ ]:
builds = get_builds_lanes()
json_object = json.dumps(builds, indent = 2) 
with open("data_builds.json", "w") as outfile:
    outfile.write(json_object)

# Código para comprobar la cantidad de partidas 

A partir de contar los míticos podemos saber la cantidad de partidas jugadas:

In [ ]:
minCount = 100000000000
lower = 5000
lowerCount = 0
temp = 0
for i in list(item_rec.keys()):
    for it in list(item_rec[i]['lane'].keys()):
        if item_rec[i]['lane'][it] != None:
            it_temp = item_rec[i]['lane'][it]
            for jt in it_temp['mythic']:
                temp += jt[1]
            if temp < minCount:
                print("MIN que", lower, ":",item_rec[i]['name'], it)
                minCount = temp
            if temp <= lower:
                print("Menos que", lower, ":",item_rec[i]['name'], it)
                lowerCount += 1
                
            temp = 0
print(minCount)
print("Menor que", lower, ":", lowerCount)